In [1]:
import os
import plaid
import requests
import datetime
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [3]:
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_SECRET = os.getenv('PLAID_SECRET')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_ENV = os.getenv('PLAID_ENV', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')

In [4]:
type(PLAID_PUBLIC_KEY)

str

# Plaid Access Token

In this section, you will use the plaid-python api to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:
1. Create a client to connect to plaid
2. Use the client to generate a public token and request the following items: 
['transactions', 'income', 'assets']
3. Exchange the public token for an access token
4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [5]:
client = plaid.Client(client_id=PLAID_CLIENT_ID, secret=PLAID_SECRET, public_key=PLAID_PUBLIC_KEY, environment='sandbox')

# Get institutions
client.Institutions.get(2)

{'institutions': [{'country_codes': ['US'],
   'credentials': [{'label': 'Username', 'name': 'username', 'type': 'text'},
    {'label': 'Password', 'name': 'password', 'type': 'password'}],
   'has_mfa': True,
   'input_spec': 'fixed',
   'institution_id': 'ins_112060',
   'mfa': ['code', 'list', 'questions', 'selections'],
   'mfa_code_type': 'numeric',
   'name': '1st Bank (Broadus, MT) - Personal',
   'oauth': False,
   'products': ['assets',
    'auth',
    'balance',
    'transactions',
    'income',
    'identity'],
   'routing_numbers': []},
  {'country_codes': ['US'],
   'credentials': [{'label': 'Username', 'name': 'username', 'type': 'text'},
    {'label': 'Password', 'name': 'password', 'type': 'password'}],
   'has_mfa': True,
   'input_spec': 'fixed',
   'institution_id': 'ins_112062',
   'mfa': ['code', 'list', 'questions', 'selections'],
   'mfa_code_type': 'numeric',
   'name': '1st Constitution Bank - Personal',
   'oauth': False,
   'products': ['assets',
    'auth',


### 2. Generate a public token

In [6]:
# Select an institution for processing
INSTITUTION_ID = "ins_109508"

### 3. Exchange the public token for an access token

In [7]:
# Create public token to be exchanged for institution access token
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions','income','assets'])

# Exchange public token for access token
exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])

# Store access token as variable
access_token = exchange_response['access_token']

### 4. Fetch Accounts

In [8]:
client.Accounts.get(access_token)

{'accounts': [{'account_id': '9QanmprjZjcDrjNl64XJCyAereXWLqCRjnyLd',
   'balances': {'available': 100,
    'current': 110,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '0000',
   'name': 'Plaid Checking',
   'official_name': 'Plaid Gold Standard 0% Interest Checking',
   'subtype': 'checking',
   'type': 'depository'},
  {'account_id': 'vZwVozXJkJfqdzEjax6bfMb9g986xdTWrlej5',
   'balances': {'available': 200,
    'current': 210,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '1111',
   'name': 'Plaid Saving',
   'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
   'subtype': 'savings',
   'type': 'depository'},
  {'account_id': 'RvagpL3A9ACD1bQJW9yqCj5xKxGXNbiR8Jk9n',
   'balances': {'available': None,
    'current': 1000,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '2222',
   'name': 'Plaid CD',
   'officia

In [10]:
client.Income.get(access_token)

{'income': {'income_streams': [{'confidence': 0.99,
    'days': 690,
    'monthly_income': 500,
    'name': 'UNITED AIRLINES'}],
  'last_year_income': 6000,
  'last_year_income_before_tax': 7285,
  'max_number_of_overlapping_income_streams': 1,
  'number_of_income_streams': 1,
  'projected_yearly_income': 6085,
  'projected_yearly_income_before_tax': 7389},
 'request_id': 'AjVOd1iZwqS0dHa'}

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [11]:
# Get transactions for institution for specific date range
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

# start_d = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-365))
# end_d = '{:%Y-%m-%d}'.format(datetime.datetime.now())
# name = 0
# amount = 0
# category = 0

# Get transactions for date range
transaction_response = client.Transactions.get(access_token,start_date,end_date)

#do income response 
income_response = client.Income.get(access_token)


### 2. Print the categories for each transaction

In [12]:
# Print JSON output
json.dumps(transaction_response['transactions'][:1],indent=4, sort_keys=True)


'[\n    {\n        "account_id": "6j3WGMl474u4ojB5VW7DT7bG1GQpWktg4GK6N",\n        "account_owner": null,\n        "amount": 500,\n        "authorized_date": null,\n        "category": [\n            "Food and Drink",\n            "Restaurants"\n        ],\n        "category_id": "13005000",\n        "date": "2020-04-25",\n        "iso_currency_code": "USD",\n        "location": {\n            "address": null,\n            "city": null,\n            "country": null,\n            "lat": null,\n            "lon": null,\n            "postal_code": null,\n            "region": null,\n            "store_number": null\n        },\n        "name": "Tectra Inc",\n        "payment_channel": "in store",\n        "payment_meta": {\n            "by_order_of": null,\n            "payee": null,\n            "payer": null,\n            "payment_method": null,\n            "payment_processor": null,\n            "ppd_id": null,\n            "reason": null,\n            "reference_number": null\n      

### 3. Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

(For categories with more than one label, just use the first category label in the list)

In [13]:
# Create Empty Lists
transactions_date=[]
transactions_name=[]
transactions_amount=[]
transactions_category=[]




In [14]:
# #append items into list
for i in transaction_response['transactions']:
    transactions_category.append((i['category'][0]))
#     transactions_name.append((i['name']))
#     transactions_amount.append((i['amount']))
#     transactions_date.append((i['date']))

# # type(transactions_category)

In [15]:
print(transactions_date)
print(transactions_amount)
print(transactions_category)
print(transactions_name)



[]
[]
['Food and Drink', 'Payment', 'Food and Drink', 'Shops', 'Payment', 'Travel', 'Transfer', 'Transfer', 'Recreation', 'Travel', 'Food and Drink', 'Food and Drink', 'Food and Drink', 'Transfer', 'Travel', 'Travel', 'Food and Drink', 'Payment', 'Food and Drink', 'Shops', 'Payment', 'Travel', 'Transfer', 'Transfer', 'Recreation', 'Travel', 'Food and Drink', 'Food and Drink', 'Food and Drink', 'Transfer', 'Travel', 'Travel', 'Food and Drink', 'Payment', 'Food and Drink', 'Shops', 'Payment', 'Travel', 'Transfer', 'Transfer', 'Recreation', 'Travel', 'Food and Drink', 'Food and Drink', 'Food and Drink', 'Transfer', 'Travel', 'Travel', 'Food and Drink']
[]


In [16]:
for i in transaction_response['transactions']:
#     print (i['name'])
    transactions_name.append((i['name']))
print(transactions_name)

['Tectra Inc', 'AUTOMATIC PAYMENT - THANK', 'KFC', 'Madison Bicycle Shop', 'CREDIT CARD 3333 PAYMENT *//', 'Uber', 'ACH Electronic CreditGUSTO PAY 123456', 'CD DEPOSIT .INITIAL.', 'Touchstone Climbing', 'United Airlines', "McDonald's", 'Starbucks', 'SparkFun', 'INTRST PYMNT', 'United Airlines', 'Uber', 'Tectra Inc', 'AUTOMATIC PAYMENT - THANK', 'KFC', 'Madison Bicycle Shop', 'CREDIT CARD 3333 PAYMENT *//', 'Uber', 'ACH Electronic CreditGUSTO PAY 123456', 'CD DEPOSIT .INITIAL.', 'Touchstone Climbing', 'United Airlines', "McDonald's", 'Starbucks', 'SparkFun', 'INTRST PYMNT', 'United Airlines', 'Uber', 'Tectra Inc', 'AUTOMATIC PAYMENT - THANK', 'KFC', 'Madison Bicycle Shop', 'CREDIT CARD 3333 PAYMENT *//', 'Uber', 'ACH Electronic CreditGUSTO PAY 123456', 'CD DEPOSIT .INITIAL.', 'Touchstone Climbing', 'United Airlines', "McDonald's", 'Starbucks', 'SparkFun', 'INTRST PYMNT', 'United Airlines', 'Uber', 'Tectra Inc']


In [17]:
# #Append name into list
# for i in transaction_response['transactions']:
#     transactions_name.append(json.dumps(i['name']))
# print(transactions_name)

In [18]:
#Append amount into list
for i in transaction_response['transactions']:
    transactions_amount.append((i['amount']))
print(transactions_amount)

[500, 2078.5, 500, 500, 25, 5.4, 5850, 1000, 78.5, -500, 12, 4.33, 89.4, -4.22, 500, 6.33, 500, 2078.5, 500, 500, 25, 5.4, 5850, 1000, 78.5, -500, 12, 4.33, 89.4, -4.22, 500, 6.33, 500, 2078.5, 500, 500, 25, 5.4, 5850, 1000, 78.5, -500, 12, 4.33, 89.4, -4.22, 500, 6.33, 500]


In [19]:
#Append date into list
for i in transaction_response['transactions']:
    transactions_date.append((i['date']))
print(transactions_date)

['2020-04-25', '2020-04-24', '2020-04-24', '2020-04-24', '2020-04-15', '2020-04-15', '2020-04-14', '2020-04-14', '2020-04-13', '2020-04-13', '2020-04-12', '2020-04-12', '2020-04-11', '2020-04-10', '2020-03-31', '2020-03-29', '2020-03-26', '2020-03-25', '2020-03-25', '2020-03-25', '2020-03-16', '2020-03-16', '2020-03-15', '2020-03-15', '2020-03-14', '2020-03-14', '2020-03-13', '2020-03-13', '2020-03-12', '2020-03-11', '2020-03-01', '2020-02-28', '2020-02-25', '2020-02-24', '2020-02-24', '2020-02-24', '2020-02-15', '2020-02-15', '2020-02-14', '2020-02-14', '2020-02-13', '2020-02-13', '2020-02-12', '2020-02-12', '2020-02-11', '2020-02-10', '2020-01-31', '2020-01-29', '2020-01-26']


In [20]:
account_df = pd.DataFrame()

In [21]:
# transactions_data = {'name': transactions_name, 'category': transactions_category, 'amount': transactions_amount,  'date': transactions_date}
# transactions_data_df = pd.DataFrame(data = transactions_data)
# transactions_data_df

,name,category,amount,date
0,Tectra Inc,Food and Drink,500.00,2020-04-25
1,AUTOMATIC PAYMENT - THANK,Payment,2078.50,2020-04-24
2,KFC,Food and Drink,500.00,2020-04-24
3,Madison Bicycle Shop,Shops,500.00,2020-04-24
4,CREDIT CARD 3333 PAYMENT *//,Payment,25.00,2020-04-15
5,Uber,Travel,5.40,2020-04-15
6,ACH Electronic CreditGUSTO PAY 123456,Transfer,5850.00,2020-04-14
7,CD DEPOSIT .INITIAL.,Transfer,1000.00,2020-04-14
8,Touchstone Climbing,Recreation,78.50,2020-04-13
9,United Airlines,Travel,-500.00,2020-04-13


In [ ]:
test_df = pd.DataFrame()

In [ ]:
#create an empty dataframe
account_df = pd.DataFrame([transactions_name,transactions_category, transactions_amount, transactions_date])
account_df

In [ ]:
account_df

In [ ]:
results = account_df.transpose()
results.head()

In [ ]:
results.columns = ['Name', "Category", 'Amount', 'Date']
# print(results)



In [ ]:
#Put Name as a string
results['Name'] = results['Name'].astype('str')

In [ ]:
#Put category as a string
results['Category'] = results['Category'].astype('str')

In [ ]:
#Put amount as a float
results['Amount'] = results['Amount'].astype('float')

In [ ]:
results['Date'] = results['Date'].str.replace("",'')
results['Date'] = results['Date'].astype('str')


# results['Category'] = results['Category'].str.replace("",'')
# results['Date'].str.replace(" ",'-')


In [ ]:
#Put date in datetimeformat
results['Date'] = results['Date'].apply(pd.to_datetime)

In [ ]:
# results_df = pd.DataFrame(results)
results.head()

In [ ]:
results.groupby('Category').Amount.sum()

### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [ ]:
results.dtypes

---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [ ]:
# Print JSON output
json.dumps(income_response['income'],indent=4, sort_keys=True)


In [ ]:
type(income_response)

In [ ]:
pretty_print_response(client.Income.get(access_token)['income'])

In [ ]:
previous_income = client.Income.get(access_token)['income']['last_year_income']
print(previous_income)

In [ ]:
current_monthly_income = client.Income.get(access_token)['income']['income_streams'][0]['monthly_income']
print(current_monthly_income)

In [ ]:
projected_income = client.Income.get(access_token)['income']['projected_yearly_income']
print(projected_income)

---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calculate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:

print(transactions_category)
type(transactions_category)

In [ ]:
transactions_data_df.groupby('category').amount.max()

In [ ]:
count = transactions_category.count('Food and Drink')
print(count)

In [ ]:
count = transactions_category.count('Payment')
print(count)

In [ ]:
count = transactions_category.count('Shops')
print(count)

In [ ]:
count = transactions_category.count('Travel')
print(count)

In [ ]:
count = transactions_category.count('Transfer')
print(count)

In [ ]:
count = transactions_category.count('Recreation')
print(count)

In [ ]:
results.groupby('Category').Amount.sum()

In [ ]:
results_bar = results.groupby('Category').Amount.count()
results_bar

In [ ]:
# results_bar.plot(kind='hist', bins=10)
results_bar.plot.bar() 

In [ ]:
# # # Create a pie chart
# plt.pie(results_bar, labels=results_bar['Category'])
# # View the plot drop above
# plt.axis('equal')
# # View the plot
# plt.show()

In [ ]:
# labels = results.index
# colors = ['lightskyblue', 'red', 'blue', 'green', 'gold','yellow']
# plt.pie(results['Category'], labels= labels, colors=colors, startangle=90, autopct='%.1f%%')
# plt.show()

### Calculate the expenses per month

In [22]:
transactions_data_df.head()

,name,category,amount,date
0,Tectra Inc,Food and Drink,500.0,2020-04-25
1,AUTOMATIC PAYMENT - THANK,Payment,2078.5,2020-04-24
2,KFC,Food and Drink,500.0,2020-04-24
3,Madison Bicycle Shop,Shops,500.0,2020-04-24
4,CREDIT CARD 3333 PAYMENT *//,Payment,25.0,2020-04-15


In [23]:
grp_date = transactions_data_df.groupby('date').amount.sum()

In [24]:
grp_date

date
2020-01-26     500.00
2020-01-29       6.33
2020-01-31     500.00
2020-02-10      -4.22
2020-02-11      89.40
2020-02-12      16.33
2020-02-13    -421.50
2020-02-14    6850.00
2020-02-15      30.40
2020-02-24    3078.50
2020-02-25     500.00
2020-02-28       6.33
2020-03-01     500.00
2020-03-11      -4.22
2020-03-12      89.40
2020-03-13      16.33
2020-03-14    -421.50
2020-03-15    6850.00
2020-03-16      30.40
2020-03-25    3078.50
2020-03-26     500.00
2020-03-29       6.33
2020-03-31     500.00
2020-04-10      -4.22
2020-04-11      89.40
2020-04-12      16.33
2020-04-13    -421.50
2020-04-14    6850.00
2020-04-15      30.40
2020-04-24    3078.50
2020-04-25     500.00
Name: amount, dtype: float64

In [25]:
for date, amount in grp_date.iteritems():
   
    
    
    month = date.split('-')[1]
 
    print('for month:{} we made:{}'.format(month, amount))

for month:01 we made:500.0
for month:01 we made:6.33
for month:01 we made:500.0
for month:02 we made:-4.22
for month:02 we made:89.4
for month:02 we made:16.33
for month:02 we made:-421.5
for month:02 we made:6850.0
for month:02 we made:30.4
for month:02 we made:3078.5
for month:02 we made:500.0
for month:02 we made:6.33
for month:03 we made:500.0
for month:03 we made:-4.22
for month:03 we made:89.4
for month:03 we made:16.33
for month:03 we made:-421.5
for month:03 we made:6850.0
for month:03 we made:30.4
for month:03 we made:3078.5
for month:03 we made:500.0
for month:03 we made:6.33
for month:03 we made:500.0
for month:04 we made:-4.22
for month:04 we made:89.4
for month:04 we made:16.33
for month:04 we made:-421.5
for month:04 we made:6850.0
for month:04 we made:30.4
for month:04 we made:3078.5
for month:04 we made:500.0


In [31]:
def get_month(stringg):
    month = stringg.split('-')[1]
    return month

In [32]:
jan_month = 0

for date, amount in grp_date.iteritems():
    month = get_month(date)
    if month == '01':
        jan_month = jan_month + amount
print(jan_month)
    

1006.3299999999999


In [1]:
feb_month = 0

for date, amount in grp_date.iteritems():
    month = get_month(date)
    if month == '02':
        feb_month = feb_month + amount
print(feb_month)
    

NameError: name 'grp_date' is not defined

In [ ]:
march_month = 0

for date, amount in grp_date.iteritems():
    month = get_month(date)
    if month == '02':
        march_month = march_month + amount
print(feb_month)
    

In [ ]:
arpil_month = 0

for date, amount in grp_date.iteritems():
    month = get_month(date)
    if month == '02':
        arpil_month = arpil_month + amount
print(arpil_month)
    